In [2]:
#Name
ModelName = 'titanic_propensity_survive'
# ATENÇÃO: nome do bucket criado no S3 (altere para o bucket com o seu nome)
bucketName = 'aula-deploy-modelos-seu-nome'

In [3]:
try:
    import sys
    sys.path.append('/var/lang/lib/python37.zip')
    sys.path.append('/var/lang/lib/python3.7')
    sys.path.append('/var/lang/lib/python3.7/lib-dynload')
    sys.path.append('/var/lang/lib/python3.7/site-packages')
    sys.path.remove('/opt/.sagemakerinternal/conda/lib/python3.7/site-packages')
except:
    pass

In [4]:
import boto3
import json
import awswrangler as wr
from decimal import Decimal
import time
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import awswrangler as wr
from pysparkling.ml import H2OMOJOSettings
from pysparkling.ml import H2OMOJOModel
from sagemaker import get_execution_role

In [5]:
my_boto3_session = boto3.Session(region_name='us-east-1')

## 1. Criar o contexto Spark

In [6]:
print(pyspark)

<module 'pyspark' from '/var/lang/lib/python3.7/site-packages/pyspark/__init__.py'>


In [7]:
#Confirmar se o camimho abaixo estão igual ao print acima. Se não estiver altere para o valor do print
findspark.init('/var/lang/lib/python3.7/site-packages/pyspark')
sc = SparkContext('local', 'SparkContext')
spark = SparkSession.builder.appName("SparkSession").getOrCreate()
spark

22/10/21 20:20:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/21 20:20:49 WARN SparkContext: The jar file:///var/lang/lib/python3.7/site-packages/sparkling_water/sparkling_water_assembly.jar has been added already. Overwriting of added jars is not supported in the current version.


## 2. Criar um dataframe Spark usando os dados da tabela auladeploymodelos.titanic_propensity_survive

In [8]:
#Primeiro vamos criar um dataframe pandas executando uma query no Athena
query = "SELECT * FROM auladeploymodelos.titanic_propensity_survive;"
dataprep_df = wr.athena.read_sql_query(query, database="auladeploymodelos", boto3_session=my_boto3_session)
dataprep_df = dataprep_df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('missing'))
dataprep_df = dataprep_df.fillna(0)
dataprep_df

,pclass,embarked,cabine_prefix,ticket_str,nametitle,fare,sibsp,parch,age_mean,ticket_int,survived,passengerid,referencedate,partition_0
0,3,S,missing,A,Mr,7.2500,1.0,0.0,22.000000,521171.0,0,1,1912-05,train_data
1,1,C,C,PC,Mrs,71.2833,1.0,0.0,38.000000,17599.0,1,2,1912-04,train_data
2,3,S,missing,STONO,Miss,7.9250,0.0,0.0,26.000000,23101282.0,1,3,1912-06,train_data
3,1,S,C,missing,Mrs,53.1000,1.0,0.0,35.000000,113803.0,1,4,1912-04,train_data
4,3,S,missing,missing,Mrs,11.1333,0.0,2.0,27.000000,347742.0,1,9,1912-05,train_data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,3,C,missing,missing,Miss,7.2250,0.0,0.0,15.000000,2667.0,1,876,1912-07,test_data
887,3,S,missing,missing,Mr,9.8458,0.0,0.0,20.000000,7534.0,0,877,1912-07,test_data
888,3,S,missing,missing,Mr,7.8958,0.0,0.0,29.699118,349217.0,0,879,1912-07,test_data
889,3,S,missing,missing,Mr,7.8958,0.0,0.0,33.000000,349257.0,0,882,1912-07,test_data


In [9]:
#Depois vamos criar um dataframe Spark usando os dados do dataframe
datasource_titanic = spark.createDataFrame(dataprep_df)
datasource_titanic.printSchema()

root
 |-- pclass: long (nullable = true)
 |-- embarked: string (nullable = true)
 |-- cabine_prefix: string (nullable = true)
 |-- ticket_str: string (nullable = true)
 |-- nametitle: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- sibsp: double (nullable = true)
 |-- parch: double (nullable = true)
 |-- age_mean: double (nullable = true)
 |-- ticket_int: double (nullable = true)
 |-- survived: long (nullable = true)
 |-- passengerid: long (nullable = true)
 |-- referencedate: string (nullable = true)
 |-- partition_0: string (nullable = true)



## 3. Criar o código que usaremos para criar um deploy usando o Glue Jobs

In [10]:
# Após testar esse código, colocar no bloco abaixo, na parte reservada para o seu scritp de deploy

#ModelPath
PathModelMojo='./output_model/models/best/StackedEnsemble_BestOfFamily_4_AutoML_1_20221011_230015.zip'

## Selected Features: variaveis que entraram no modelo
CAT = [
'pclass'
,'embarked'
,'cabine_prefix'
,'ticket_str'
,'nametitle']

#float
NUM = [
'fare'
,'sibsp'
,'parch'
,'age_mean'
,'ticket_int']
selected_features = CAT + NUM

##functions
def ratings(p1):
    if p1 <= 0.2508362656036639:
        return 1
    elif p1 <= 0.6540492277407066:
        return 2
    else:
        return 3
ratings_func = F.udf(ratings, StringType())

def predict_survive(predict):
    if predict == 0:
        return 'Not survive'
    elif predict == 1:
        return 'Survive'
    else:
        return 'predict_ERROR'
predict_func = F.udf(predict_survive, StringType())

## Escorar a base, criar o score e o rating
settings = H2OMOJOSettings(convertUnknownCategoricalLevelsToNa = True, convertInvalidNumbersToNa = True)
model = H2OMOJOModel.createFromMojo(PathModelMojo, settings)
predict_titanic_1 = model.transform(datasource_titanic)

predict_titanic = predict_titanic_1.withColumn('predict_int', F.col('detailed_prediction.label').cast(IntegerType()))\
.withColumn('probability', F.col('detailed_prediction.probabilities.1'))\
.withColumn('rating', ratings_func((F.col('probability'))))\
.withColumn('predict', predict_func(F.col('predict_int')))\
.drop('detailed_prediction', 'prediction', 'predict_int', 'partition_0')

first_cols = ['pclass', 'embarked', 'cabine_prefix', 'ticket_str', 'nametitle']
other_cols = sorted([c for c in predict_titanic.columns if c not in first_cols])
rearanged_cols = first_cols + other_cols
predict_titanic = predict_titanic.select(rearanged_cols)

## 4. Alterar o código final que será executado no Glue Jobs. 
#### ATENÇÃO: Não executar esse código, pois vai dar erro. Ele irá funcionar somente no Glue Jobs

In [17]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.context import SparkContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pysparkling.ml import H2OMOJOSettings
from pysparkling.ml import H2OMOJOModel
import pandas as pd

# @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ["JOB_NAME"])

#Job setup
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

job = Job(glueContext)
job.init(args["JOB_NAME"], args)

datasource_titanic = glueContext.create_dynamic_frame.from_catalog(database = "auladeploymodelos", table_name = "titanic_propensity_survive", transformation_ctx = "datasource_titanic_propensity_survive", additional_options={"mergeSchema": "true"}).toDF()\
.fillna(value="Missing")

#Name
ModelName = 'titanic_propensity_survive'
# ATENÇÃO: nome do bucket criado no S3 (altere para o bucket com o seu nome)
bucketName = 'aula-deploy-modelos-seu-nome'

#ATENÇÃO
#Remover ./output_model/models/best/ da variável PathModelMojo e deixar somente o nome do arquivo .zip



############################################ COLAR O CÓDIGO DO DEPLOY SOMENTE ABAIXO DESSA LINHA ###########################

#ModelPath
PathModelMojo='StackedEnsemble_BestOfFamily_4_AutoML_1_20221011_230015.zip'

## Selected Features: variaveis que entraram no modelo
CAT = [
'pclass'
,'embarked'
,'cabine_prefix'
,'ticket_str'
,'nametitle']

#float
NUM = [
'fare'
,'sibsp'
,'parch'
,'age_mean'
,'ticket_int']
selected_features = CAT + NUM

##functions
def ratings(p1):
    if p1 <= 0.2508362656036639:
        return 1
    elif p1 <= 0.6540492277407066:
        return 2
    else:
        return 3
ratings_func = F.udf(ratings, StringType())

def predict_survive(predict):
    if predict == 0:
        return 'Not survive'
    elif predict == 1:
        return 'Survive'
    else:
        return 'predict_ERROR'
predict_func = F.udf(predict_survive, StringType())

## Escorar a base, criar o score e o rating
settings = H2OMOJOSettings(convertUnknownCategoricalLevelsToNa = True, convertInvalidNumbersToNa = True)
model = H2OMOJOModel.createFromMojo(PathModelMojo, settings)
predict_titanic_1 = model.transform(datasource_titanic)

predict_titanic = predict_titanic_1.withColumn('predict_int', F.col('detailed_prediction.label').cast(IntegerType()))\
.withColumn('probability', F.col('detailed_prediction.probabilities.1'))\
.withColumn('rating', ratings_func((F.col('probability'))))\
.withColumn('predict', predict_func(F.col('predict_int')))\
.drop('detailed_prediction', 'prediction', 'predict_int', 'partition_0')

first_cols = ['pclass', 'embarked', 'cabine_prefix', 'ticket_str', 'nametitle']
other_cols = sorted([c for c in predict_titanic.columns if c not in first_cols])
rearanged_cols = first_cols + other_cols
predict_titanic = predict_titanic.select(rearanged_cols)

############################################# FIM DO BLOCO PARA COLOCAR SEU CÓDIGO #############################################



## Salvar no Lakehouse no format parquet
predict_titanic.write.mode("overwrite").format("parquet").partitionBy('embarked', 'pclass').save('s3://%s/databases/%s' % (bucketName, ModelName+'_spark'))

job.commit()

#fim

### 4.1 Copiar e colar o conteúdo inteiro do bloco a cima e salvar no arqwuivo ./deploy_glue/titanic_predict_glue.py

## 5. Fazer upload para o S3 dos arquivos que serão usados pelo Glue jobs

In [12]:
s3_resource = my_boto3_session.resource('s3')
s3_resource.meta.client.upload_file(PathModelMojo, bucketName, f'deploy/{PathModelMojo.split("/")[-1]}')
s3_resource.meta.client.upload_file('./deploy_glue/sparkling-water-assembly-scoring_2.12-3.36.1.1-1-3.1-all.jar', bucketName, 'deploy/sparkling-water-assembly-scoring_2.12-3.36.1.1-1-3.1-all.jar')
s3_resource.meta.client.upload_file('./deploy_glue/titanic_predict_glue.py', bucketName, 'deploy/titanic_predict_glue.py')

## 6. Criar e executar o Glue Job

In [13]:
glue = my_boto3_session.client('glue', region_name="us-east-1")

#Excluir o job antes de criar caso ele já exista
response = glue.delete_job(
    JobName='TitanicJob'
)

#Criar o job com o script definido em titanic_predict_glue.py
response_create = glue.create_job(
    Name='TitanicJob',    
    Role= get_execution_role(),
    Command={
        'Name': 'glueetl',
        'ScriptLocation': f's3://{bucketName}/deploy/titanic_predict_glue.py',
        'PythonVersion': '3'
    },
    DefaultArguments={
      '--TempDir': f's3://{bucketName}/deploy',
      '--job-bookmark-option': 'job-bookmark-disable',
      '--additional-python-modules':   'h2o-pysparkling-3.1==3.36.1.3.post1',
      '--extra-jars':  f's3://{bucketName}/deploy/sparkling-water-assembly-scoring_2.12-3.36.1.1-1-3.1-all.jar',
      '--extra-files':  f's3://{bucketName}/deploy/{PathModelMojo.split("/")[-1]}'
    },
    MaxRetries=0,
    GlueVersion='3.0',
    NumberOfWorkers=2,
    WorkerType='Standard',
      Tags={
          'cost_by_tags': 'glue-jobs-titanic'
      },
)

print(json.dumps(response_create, indent=4, sort_keys=True, default=str))

{
    "Name": "TitanicJob",
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "21",
            "content-type": "application/x-amz-json-1.1",
            "date": "Fri, 21 Oct 2022 20:21:14 GMT",
            "x-amzn-requestid": "f944ef9e-7390-49dd-9521-21ee981cd852"
        },
        "HTTPStatusCode": 200,
        "RequestId": "f944ef9e-7390-49dd-9521-21ee981cd852",
        "RetryAttempts": 0
    }
}


In [14]:
response_run = glue.start_job_run(
    JobName=response_create['Name']
)
response_run

{'JobRunId': 'jr_0bb94379c0f54e506b6b1191062930cf5c0de769b22766fca308f71b4f2fca62',
 'ResponseMetadata': {'RequestId': '589feb13-a40a-4408-9a2d-06baf3503b72',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 21 Oct 2022 20:21:14 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'connection': 'keep-alive',
   'x-amzn-requestid': '589feb13-a40a-4408-9a2d-06baf3503b72'},
  'RetryAttempts': 0}}

In [15]:
#Aguardar até o job ser concluido
JobRunState='RUNNING'

while JobRunState == 'RUNNING':    
    response_get = glue.get_job_run(
        JobName=response_create['Name'],
        RunId=response_run['JobRunId'],
        PredecessorsIncluded=True
    )
    JobRunState = response_get['JobRun']['JobRunState']
    print('Job Status: ' + JobRunState)
    time.sleep(20)

Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: SUCCEEDED


In [16]:
#Usar o mesmo crawler criado no código 1_Data_Prep.ipynb para adiconar a nova tablea
glue.update_crawler(Name='aula-deploy-modelos',
                    Targets={'S3Targets': [{'Path': 's3://%s/databases/%s' % (bucketName, ModelName+'_spark'),
                               'Exclusions': []}]})

glue.start_crawler(Name='aula-deploy-modelos')

{'ResponseMetadata': {'RequestId': '9668d3d7-4529-43db-a06c-12982c4cccf5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 21 Oct 2022 20:23:38 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9668d3d7-4529-43db-a06c-12982c4cccf5'},
  'RetryAttempts': 0}}

## 7. Verificar o resultado no Athena
#### 7.1. Voltar para a página da AWS e em *Services*, no canto superior esquerdo procure por *Athena*
#### 7.2. Em data base, no canto superior esquerdo, selecione auladeploymodelos
#### 7.3. Clique sobre a tabela titanic_propensity_survive_spark e selecione a opção "Preview Table"
#### 7.4. Confira se os dados da tabela estão ok